# cust_info

## Read Data & Transformation From Silver Delta Table

In [0]:
# Read & Transform Data (GOLD layer)

query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY customer_id) AS customer_number,
    customer_id,
    first_name,
    last_name
FROM silver.crm_cust_info
"""

df = spark.sql(query)

df.limit(10).display()


## Writing Gold Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_cust_info")

# prod_info

## Read Data & Transformation From Silver Delta Table

In [0]:
#Read & Transform

query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY pn.start_date, pn.product_number) AS product_key, -- Surrogate key
    pn.product_id,
    pn.product_number,
    pn.product_name,
    pn.category_id,
    pn.product_line,
    pn.start_date
FROM silver.crm_prod_info pn
"""
df = spark.sql(query)

     
# Show Data
df.limit(10).display()
     


## Writing Gold Table

In [0]:


df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_prod_info")
     


# sales_info

## Read Data & Transform From  Gold Delta Table

In [0]:

#Read & Transform
query = """
SELECT
    sd.order_number,
    pr.product_key,
    cu.customer_number,
    sd.order_date,
    sd.ship_date,
    sd.due_date,
    sd.sales_amount,
    sd.quantity,
    sd.price
FROM silver.crm_sales sd
LEFT JOIN gold.dim_prod_info pr
    ON sd.product_number = pr.product_number
LEFT JOIN gold.dim_cust_info cu
    ON sd.customer_id = cu.customer_id;
"""
df = spark.sql(query)

     
#Show Data
df.limit(10).display()
     


## Write Into Gold Table 

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.fact_sales")